## Impoort data from Excel

In [3]:
#machine learning(clustering) packages
import unicodedata
import os,re
import np
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import nltk
from fuzzywuzzy import fuzz
from sklearn.cluster import AffinityPropagation
import difflib

In [83]:
#regular packages
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from cleanco import basename

In [84]:
# using pandas dataframe we read the excel file
df = pd.read_excel (io="Python data.xlsx",sheet_name='TVM Names')
df.head()

,SOURCE_SYSTEM,VENDOR_NUMBER,VENDOR_NAME,ADDRESS1,total_spend
0,IBM,416687,THE PEANUT INSTITUTE,PO BOX 70157 ...,10000.0
1,JDE 2.0,810525,PRO FLEET LLC,3932 S EL AMBER AVE ...,10000.0
2,JDE 2.0,866682,INSIGHTSENSE INC,873 GLENBARD RD ...,10000.0
3,IBM,M03055,ALZA INC,74 GRAND AVE ...,10000.0
4,IBM,633099,RESTAURANT SUPPLY CHAIN SOLUTIONS,LLC ...,10000.0


In [35]:
df = pd.read_excel (io="Python data.xlsx",sheet_name='Normalized')
df=df.rename(columns={'NORMALIZED_NAME':'VENDOR_NAME'})
df.head()

,SOURCE_SYSTEM,VENDOR_NUMBER,VENDOR_NAME
0,IBM,211417,Wells - Fastenal
1,IBM,590733,Wells - Fastenal
2,OFC_NEOVIA_LATAM,5727,Wells - Fastenal
3,SAP-POL,27162125,Wells - Fastenal
4,SAP-POL,27168406,Wells - Fastenal


In [85]:
#drop total_spend column
df=df.drop(['total_spend'],axis=1)
df=df.drop(['ADDRESS1'],axis=1)

In [88]:
#make sure the VENDOR_NAME count is matching with source_system and vendor_number after data cleaning
df.count()

SOURCE_SYSTEM    74515
VENDOR_NUMBER    74515
VENDOR_NAME      74515
dtype: int64

## Preliminary Data cleaning

In [87]:
df.astype(str)
# clean up empty & null entires for vendor names
df['VENDOR_NAME'].replace(r'', np.NaN)
df.dropna(subset=['VENDOR_NAME'],inplace=True)

In [74]:
#make sure no null value
df[df.VENDOR_NAME.isnull()]

,SOURCE_SYSTEM,VENDOR_NUMBER,VENDOR_NAME,ADDRESS1,total_spend


In [11]:
#remove leadinga and trailing space
df['VENDOR_NAME']=df['VENDOR_NAME'].str.strip()
df['VENDOR_NAME']=df['VENDOR_NAME'].str.lstrip()
df.count()

SOURCE_SYSTEM    74515
VENDOR_NUMBER    74515
VENDOR_NAME      74515
dtype: int64

In [89]:
df = df.sort_values('VENDOR_NAME',ignore_index=True)
df.tail()

,SOURCE_SYSTEM,VENDOR_NUMBER,VENDOR_NAME
74510,WILD-EU,5222945,knip GmbH und Co.KG
74511,ANI,57759,m2 commodities llc
74512,ANI,57707,naimko Plastics & Trophy Center
74513,ANI,58052,steam valley farms
74514,WILD-EU,5212116,von Oertzen GmbH


## Students' solution (as a comparison)

In [152]:
def calculateMatch(name1, name2):
    regularRatio = fuzz.ratio(name1, name2)
    partialRatio = fuzz.partial_ratio(name1, name2)
    maxMatchRatio = fuzz.token_sort_ratio(name1, name2)
    return [regularRatio, partialRatio, maxMatchRatio]

def normalize_names(vendorNamesList):

    # set the root name, updated if not matched
    rootVendorName = vendorNamesList[0]

    # initialize 2 lists to keep track of normalized and percent confidence
    normalizedList = []
    percentsList = []

    # iterate through names in vendor name list from first index
    for i in range(1, len(vendorNamesList)):

        previousName = vendorNamesList[i - 1]
        currentName = vendorNamesList[i]

        # calculate the percent match
        percentMatch = calculateMatch(currentName, previousName)[2]

        # if the names match is over 80%, normalize the previous one
        if percentMatch > 65:

            # add the root to normalized names list
            normalizedList.append(rootVendorName)

        # if match is less than or equal to 80%
        else:

            # add the previous root name to normalized names list
            normalizedList.append(rootVendorName)

            # update the root vendor name
            rootVendorName = vendorNamesList[i]

        # for each comparison, add the percent match (confidence)
        percentsList.append(percentMatch)

    # percent match for last comparison would naturally be zero
    percentsList.append(0)

    # add the last name to the normalized names list
    normalizedList.append(vendorNamesList[-1])

    return normalizedList, percentsList

In [153]:
# list of original vendor names
vendorNamesList = []

# converting names of vendors from pandas dataframe to a list
vendorNamesList=df['VENDOR_NAME'].tolist()

# iterate over the vendor name list
for i in range(len(vendorNamesList)):

    # vendor name from current iteration
    name = vendorNamesList[i]

    # some vendor names are empty, null, or cannot be parsed as strings
    if type(name) is not str:
        name = str(name)

    # remove trailing spaces from the vendor names
    vendorNamesList[i] = name.rstrip()

normalList, percentList = normalize_names(vendorNamesList)
normalList, percentList = normalize_names(normalList)

In [154]:
df['Normalized Name(Students)']=normalList
df['Percent Match(Students)']=percentList

## Normalization with Cleanco

In [90]:
#New column for normalized name, all Cap
df['Supplier_Name_Normalized']=df.VENDOR_NAME.str.upper()

### ADM Customized Logics

In [91]:
# Specific Supplier Names Rules
df.loc[df.Supplier_Name_Normalized.astype(str).str.contains("ADM "),'Supplier_Name_Normalized']= 'ADM'
df.loc[df.Supplier_Name_Normalized.astype(str).str.contains("ARCHER DANIELS MIDLAND"),'Supplier_Name_Normalized']= 'ADM'
df.loc[df.Supplier_Name_Normalized.astype(str).str.contains("MICROSOFT"),'Supplier_Name_Normalized']= 'MICROSOFT'
df.loc[df.Supplier_Name_Normalized.astype(str).str.contains("AT&T"),'Supplier_Name_Normalized']= 'ATT'
df.loc[df.Supplier_Name_Normalized.astype(str).str.contains("A T & T"),'Supplier_Name_Normalized']= 'ATT'
df.loc[df.Supplier_Name_Normalized.astype(str).str.startswith("DELL "),'Supplier_Name_Normalized']= 'DELL'
df.loc[df.Supplier_Name_Normalized.astype(str).str.contains("FASTENAL "),'Supplier_Name_Normalized']= 'WELLS - FASTENAL'
df.loc[df.Supplier_Name_Normalized.str.startswith("EXXON"),'Supplier_Name_Normalized']='EXXON MOBIL'
# Remove commas
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.astype(str).str.replace(',', '',regex=True)
# Remove hyphens
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.astype(str).str.replace(' - ', ' ',regex=True)
# Remove &
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.astype(str).str.replace(' & ',' ',regex=True)
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.astype(str).str.replace('&',' ',regex=True)
# Remove very specific intl abbr 
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.astype(str).str.replace('S.A.L.', '',regex=True)
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.astype(str).str.replace('S.A.R.L.', '',regex=True)
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.astype(str).str.replace('CO.,LTD', '',regex=True)
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.astype(str).str.replace("INT'L", '',regex=True)
#REMOVE END TEXT
df.loc[df.Supplier_Name_Normalized.astype(str).str.endswith(' S A'),'Supplier_Name_Normalized']= df[df.Supplier_Name_Normalized.astype(str).str.endswith(' S A')].Supplier_Name_Normalized.astype(str).str.replace(' S A', '',regex=True)
df.loc[df.Supplier_Name_Normalized.astype(str).str.endswith(' S/A'),'Supplier_Name_Normalized']= df[df.Supplier_Name_Normalized.astype(str).str.endswith(' SA')].Supplier_Name_Normalized.astype(str).str.replace(' SA', '',regex=True)
df.loc[df.Supplier_Name_Normalized.astype(str).str.endswith(' B V'),'Supplier_Name_Normalized']= df[df.Supplier_Name_Normalized.astype(str).str.endswith(' B V')].Supplier_Name_Normalized.astype(str).str.replace(' B V', '',regex=True)
df.loc[df.Supplier_Name_Normalized.astype(str).str.endswith(' SALES LLC'),'Supplier_Name_Normalized']= df[df.Supplier_Name_Normalized.astype(str).str.endswith(' SALES LLC')].Supplier_Name_Normalized.astype(str).str.replace(' SALES LLC', '',regex=True)
#REMOVE ALL TEXT AFTER A STRING
df['Supplier_Name_Normalized'] = df['Supplier_Name_Normalized'].str.split('LTDA').str[0]
df['Supplier_Name_Normalized'] = df['Supplier_Name_Normalized'].str.split('SP.Z.O.O.').str[0]
df['Supplier_Name_Normalized'] = df['Supplier_Name_Normalized'].str.split('SP ZOO').str[0]
df['Supplier_Name_Normalized'] = df['Supplier_Name_Normalized'].str.split('S A C I').str[0]
df['Supplier_Name_Normalized'] = df['Supplier_Name_Normalized'].str.split('SA DE CV').str[0]
df['Supplier_Name_Normalized'] = df['Supplier_Name_Normalized'].str.split('DO BRASIL').str[0]
df['Supplier_Name_Normalized'] = df['Supplier_Name_Normalized'].str.split(' INTERNATIONAL').str[0]
# Remove text between parenthesis 
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.astype(str).str.replace(r"\(.*\)","",regex=True)

In [129]:
df[df.VENDOR_NAME.astype(str).str.contains("S A C I")]

,SOURCE_SYSTEM,VENDOR_NUMBER,VENDOR_NAME,Supplier_Name_Normalized
10867,JSA,10141975,CARGILL AGROPECUARIA S A C I,CARGILL AGROPECUARIA
58536,JSA,10134304,RIEDER CIA S A C I,RIEDER CIA
74493,JSA,10137478,ZUSA S A C I,ZUSA


### Clean with CleanCo package

In [92]:
#clean 1st round
df.Supplier_Name_Normalized=df.Supplier_Name_Normalized.apply(lambda x: basename(x) if type(x)==str else x)
# Remove dots
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.astype(str).str.replace('.', '',regex=True)
#clean 2st round
df.Supplier_Name_Normalized=df.Supplier_Name_Normalized.apply(lambda x: basename(x) if type(x)==str else x)

In [93]:
#write to a new excel sheet
df.to_excel("output.xlsx",sheet_name='Normalized Name')

## Address Matching

In [139]:
df[df.duplicated(subset=['ADDRESS1'])==True&df['ADDRESS1'].notnull()]

,SOURCE_SYSTEM,VENDOR_NUMBER,VENDOR_NAME,ADDRESS1,Supplier_Name_Normalized
2,JDE 2.0,6008,05-Corporate,NaN,05-CORPORATE
116,IBM,482567,1776 TRANSPORTATION LLC,846 5TH ST ...,1776 TRANSPORTATION
118,JDE 2.0,890215,18 WHEELER TRANSPORTATION INC,8132 SUNLAND BLVD STE C ...,18 WHEELER TRANSPORTATION
157,JDE 2.0,850885,21 OAKS LLC,8217 S 1180 W ...,21 OAKS
166,IBM,G11233,2219811 AB LTD,PO BOX 1110 ...,2219811
...,...,...,...,...,...
74481,IBM,Z99033,ZUMBRO RIVER BRAND INC,1215 HERSHEY ST ...,ZUMBRO RIVER BRAND
74485,JDE 2.0,196316,ZUPA GLOBAL INC,DBA DSL EXPRESS ...,ZUPA GLOBAL
74492,JDE 2.0,354767,ZURICH NORTH AMERICA,8745 PAYSPHERE CIRCLE ...,ZURICH NORTH AMERICA
74496,IBM,183493,ZYLSTRA BROKERAGE INC,PO BOX 368 ...,ZYLSTRA BROKERAGE


In [167]:
df[df.duplicated(subset=['ADDRESS1'])==True&df['ADDRESS1'].notnull()&(df['ADDRESS1'].str.len()>2)]

,SOURCE_SYSTEM,VENDOR_NUMBER,VENDOR_NAME,ADDRESS1,Supplier_Name_Normalized
2,JDE 2.0,6008,05-Corporate,NaN,05-CORPORATE
116,IBM,482567,1776 TRANSPORTATION LLC,846 5TH ST ...,1776 TRANSPORTATION
118,JDE 2.0,890215,18 WHEELER TRANSPORTATION INC,8132 SUNLAND BLVD STE C ...,18 WHEELER TRANSPORTATION
157,JDE 2.0,850885,21 OAKS LLC,8217 S 1180 W ...,21 OAKS
166,IBM,G11233,2219811 AB LTD,PO BOX 1110 ...,2219811
...,...,...,...,...,...
74481,IBM,Z99033,ZUMBRO RIVER BRAND INC,1215 HERSHEY ST ...,ZUMBRO RIVER BRAND
74485,JDE 2.0,196316,ZUPA GLOBAL INC,DBA DSL EXPRESS ...,ZUPA GLOBAL
74492,JDE 2.0,354767,ZURICH NORTH AMERICA,8745 PAYSPHERE CIRCLE ...,ZURICH NORTH AMERICA
74496,IBM,183493,ZYLSTRA BROKERAGE INC,PO BOX 368 ...,ZYLSTRA BROKERAGE


In [168]:
df1=df[df.duplicated(subset=['ADDRESS1'])==True&df['ADDRESS1'].notnull()&(df['ADDRESS1'].str.len()>2)]

In [169]:
df1 = df1.sort_values('ADDRESS1',ignore_index=True)

## Machine learning with clustering supplier names

In [205]:
df=df.tail(1000)

In [173]:
# List of keywords to help identify stop_words
vendor_stopwords=['biz', 'bv', 'co', 'comp', 'company', 
                'corp','corporation', 'dba', 
                'inc', 'incorp', 'incorporat', 
                'incorporate', 'incorporated', 'incorporation', 
                'international', 'intl', 'intnl', 
                'limited' ,'llc', 'ltd', 'llp', 
                'machines', 'pvt', 'pte', 'private', 'unknown']

# Text data encoder function
def filter_ascii(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

# Remove spl characters & digits (optional) function
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

# Remove vendor specific stop words
def clean_stopwords(text,eng=False):
    if eng == False:
        custom = vendor_stopwords
    else:
        custom = vendor_stopwords + list(ENGLISH_STOP_WORDS)
    for x in custom:
        pattern2 = r'\b'+x+r'\b'
        text=re.sub(pattern2,'',text)
    return text

# Trim the text to remove double spaces
def clean_spaces(text):
    text=text.replace('  ', ' ')
    text=text.strip()
    if len(text) < 1:
        text='Tooshorttext'
    return text

# Function to Preprocess Textual data. Provide input as df['Column Name'] to this function
def preprocess_text(column, remove_digits=True, lemm=True, eng=False):
    try:
        column = [filter_ascii(text) for text in column]
        column = [remove_special_characters(text, remove_digits) for text in column]
        column = [text.lower() for text in column]
        column = [clean_stopwords(text, eng) for text in column]
        column = [clean_spaces(text) for text in column]
        ## Lemmatisation (convert the word into root word)
        if lemm == True:
            lem = nltk.stem.wordnet.WordNetLemmatizer()
            column = [lem.lemmatize(text) for text in column]
        return column
    except Exception as e:
        return print(e)

In [206]:
df['Unsupervised']=df.VENDOR_NAME.str.upper()

C:\Users\A427032\AppData\Local\Temp/ipykernel_27436/3230421696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Unsupervised']=df.VENDOR_NAME.str.upper()


In [207]:
df['Cleaned_Name']=preprocess_text(df['VENDOR_NAME'],remove_digits=False,lemm=True,eng=False)

C:\Users\A427032\AppData\Local\Temp/ipykernel_27436/3315529136.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cleaned_Name']=preprocess_text(df['VENDOR_NAME'],remove_digits=False,lemm=True,eng=False)


In [208]:
df.tail(50)

,SOURCE_SYSTEM,VENDOR_NUMBER,VENDOR_NAME,ADDRESS1,Supplier_Name_Normalized,Unsupervised,Cleaned_Name
74465,WILD-EU,5216462,ZPO Sambor Sp. z o.o.,NaN,ZPO SAMBOR,ZPO SAMBOR SP. Z O.O.,zpo sambor sp z oo
74466,JSA,10390943,ZT COMERCIAL AGRICOLA LTDA,ROD MUN LUCIANO FURIAN 3601,ZT COMERCIAL AGRICOLA,ZT COMERCIAL AGRICOLA LTDA,zt comercial agricola ltda
74467,IBM,332987,ZUBER GRAIN INC,1926 Q AVE ...,ZUBER GRAIN,ZUBER GRAIN INC,zuber grain
74468,IBM,560339,ZUBERS CORPORATION,7825 S 3500 E ...,ZUBERS,ZUBERS CORPORATION,zubers
74469,JSA,10512808,ZUBIM AMILCAR TUTUNYI ZALAZAR,LAS RESIDENTAS LA PALOMA,ZUBIM AMILCAR TUTUNYI ZALAZAR,ZUBIM AMILCAR TUTUNYI ZALAZAR,zubim amilcar tutunyi zalazar
74470,JSA,10492821,ZUCONELLI E KERBER LTDA,ROD MT 338 KM 92 SN MAIS 1000 MT A ESQ,ZUCONELLI E KERBER,ZUCONELLI E KERBER LTDA,zuconelli e kerber ltda
74471,JSA,10521458,ZUF AGRO LTDA,ROD BR 050 SN,ZUF AGRO,ZUF AGRO LTDA,zuf agro ltda
74472,JDE 2.0,264215,ZUHORN AND PARTNER,ALFREDSTRASSE 239-241 ...,ZUHORN AND PARTNER,ZUHORN AND PARTNER,zuhorn and partner
74473,WILD-EU,5208656,"ZUKAN, S.L.",NaN,ZUKAN,"ZUKAN, S.L.",zukan sl
74474,JSA,10509091,ZULEICA DE MELLO WIEZEL,SIT IPEZINHO SN,ZULEICA DE MELLO WIEZEL,ZULEICA DE MELLO WIEZEL,zuleica de mello wiezel


In [180]:
# Function to generate similarity matrix. Provide input as df['Column Name'] to this function
def fuzz_similarity(column):
  similarity_array = np.ones((len(column), (len(column))))*100
  for i in range(1, len(column)):
    for j in range(i):
      s1 = fuzz.token_set_ratio(column[i],column[j]) + 0.00000000001
      s2 = fuzz.partial_ratio(column[i],column[j]) + 0.00000000001
      similarity_array[i][j] = 2*s1*s2 / (s1+s2)
      
  for i in range(len(column)):
    for j in range(i+1,len(column)):
      similarity_array[i][j] = similarity_array[j][i]
      np.fill_diagonal(similarity_array, 100)
  return similarity_array

In [181]:
def company_clusters(dataframe, matrix):
    cust_ids = dataframe['VENDOR_NUMBER'].to_list()
    clusters = AffinityPropagation(affinity='precomputed').fit_predict(matrix)
    df_clusters = pd.DataFrame(list(zip(cust_ids, clusters)), columns=['VENDOR_NUMBER','Cluster'])
    new = pd.merge(dataframe,df_clusters, on='VENDOR_NUMBER')
    return new

In [209]:
company_clusters(df,fuzz_similarity(df['Cleaned_Name']))

KeyError: 1

In [210]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from difflib import SequenceMatcher
from collections import Counter
from sklearn.model_selection import train_test_split

# Function to merge text descriptions
def merge_desc(dataframe, column_list):
  try:
    dataframe['bow'] = dataframe[column_list].apply(lambda x: ' '.join(x), axis = 1)
  except Exception as e:
    return print(e)
  return dataframe

# Function to identify goods/services from text. Provide input as dataframe & column name
def gen_itemtype(dataframe, column_name, services_keywords_list):
    final_list = '|'.join(services_keywords_list)
    try:
      dataframe['Is_Service']=pd.np.where(dataframe[column_name].str.contains(final_list, case=True),1,0)
    except Exception as e:
      return print(e)
    return dataframe
    # dataframe['Is_Service']=pd.np.where(dataframe[column_name].str.contains(final_list, case=True),1,0)
    # return dataframe

# Function to aggregate spend. Provide input as dataframe & list of columns
def spend_agg(dataframe, columns, percentage):
    try:
        aggregated_spend=pd.DataFrame(dataframe.groupby(columns)['Total Price'].agg(['sum','count']).reset_index())
    except Exception as e:
        return print(e)
    else:
        # Sort df by count
        aggregated_spend = aggregated_spend.sort_values(by=('count'), ascending = False)
        # Creating a column for cummaltive amount of rows
        aggregated_spend['cum_sum_count'] = aggregated_spend['count'].cumsum()
        # Creating a column for cummaltive percent of rows
        aggregated_spend['cum_perc_count'] = round(100*aggregated_spend['cum_sum_count']/aggregated_spend['count'].sum(),1)
        # Creating a column for cummaltive sum of total spend
        aggregated_spend['cum_sum_sum'] = aggregated_spend['sum'].cumsum()
        # Creating a column for cummaltive percent of total spend
        aggregated_spend['cum_perc_sum'] = round(100*aggregated_spend['cum_sum_sum']/aggregated_spend['sum'].sum(),1)       
        # Spend Segregator to segregate strategic & tail spend
        aggregated_spend['spend_type'] = np.where((aggregated_spend['cum_perc_count']>float(percentage)), 'Tail Spend', 'Strategic Spend') 
    return aggregated_spend

# Function to merge dataframes
def df_merger(df_left, df_right, how, on):
  try:
    final=df_left.merge(df_right, how=how, on=on)
  except Exception as e:
    return print(e)  
  return final

# Function to group bottom 20% of spend. Must use df from spend aggregator function & item type generator function
def spend_grouper(dataframe, percentage):
    dataframe['Final_Category'] = np.where((dataframe['cum_perc_count']>float(percentage)) & (dataframe['Is_Service'] == 1), 'Other Services',
                                        np.where((dataframe['cum_perc_count']>float(percentage)) & (dataframe['Is_Service'] == 0), 'Other Goods',
                                        dataframe['Segment Title']))
    dataframe['Final_Code'] = dataframe['Final_Category'].factorize()[0]
    return dataframe

# Function to generate similarity matrix. Provide input as df['Column Name'] to this function
def fuzz_similarity(column):
  similarity_array = np.ones((len(column), (len(column))))*100
  for i in range(1, len(column)):
    for j in range(i):
      s1 = fuzz.token_set_ratio(column[i],column[j]) + 0.00000000001
      s2 = fuzz.partial_ratio(column[i],column[j]) + 0.00000000001
      similarity_array[i][j] = 2*s1*s2 / (s1+s2)
      
  for i in range(len(column)):
    for j in range(i+1,len(column)):
      similarity_array[i][j] = similarity_array[j][i]
      np.fill_diagonal(similarity_array, 100)
  return similarity_array

# Function to standardize vendor names. Provide input as dataframe to this function
def standard_name(df_clusters):
  d_standard_name = {}
  for cluster in df_clusters.Cluster.unique():
    names = df_clusters[df_clusters['Cluster']==cluster].Cleaned_Name.to_list()
    l_common_substring = []
    if len(names)>1:
      for i in range(0, len(names)):
        for j in range(i+1, len(names)):
          seqMatch = SequenceMatcher (None, names[i],names[j])
          match = seqMatch.find_longest_match(0, len(names[i]), 0, len(names[j]))
          if (match.size!=0):
            l_common_substring.append(names[i][match.a: match.a + match.size].strip())

      #n = len(l_common_substring)
      counts = Counter(l_common_substring)
      get_mode = dict(counts)
      mode = [k for k, v in get_mode.items() if v == max(list(counts.values()))]
      d_standard_name[cluster] = ";".join(mode)
    else:
      d_standard_name[cluster] = names[0]

  df_standard_names = pd.DataFrame((list(d_standard_name.items())), columns=['Cluster', 'StandardName'])
  df_clusters = df_clusters.merge(df_standard_names, on='Cluster', how='left')
  df_clusters['Score_with_standard'] = df_clusters.apply(lambda x: fuzz.token_set_ratio(x['StandardName' ],x['Cleaned_Name']),axis=1)
  df_clusters['standard_name_withoutSpaces'] = df_clusters.StandardName.apply(lambda x: x.replace(" ",""))
  for name in df_clusters.standard_name_withoutSpaces.unique():
    if len(df_clusters[df_clusters.standard_name_withoutSpaces==name].Cluster.unique()) > 1:
      df_clusters.loc[df_clusters.standard_name_withoutSpaces==name, 'StandardName'] = name
  return df_clusters.drop('standard_name_withoutSpaces', axis=1)

# Function to split independent and dependent variables from dataframe. Provide dataframe as input
def split_variables(dataframe):
  try:
    x=dataframe.drop(columns = train_target)
    y=dataframe[train_target]
    return x,y
  except Exception as e:
    print(e)

# Function to split dataframes into train & test sets. Provide dataframe of both independent and dependent variables as input
def train_test(dataframe, percentage):
  try:
    return train_test_split(dataframe, test_size=float(percentage))
  except Exception as e:
    return print(e)

# Function to generate embedding matrix. Provide vocab & word2vec model as input to the function.
def embedding_matrix(dic_vocabulary, nlp):
  ## start the matrix (length of vocabulary x vector size) with all 0s
  embeddings = np.zeros((len(dic_vocabulary)+1, 100))
  for word,idx in dic_vocabulary.items():
      ## update the row with vector
      try:
        embeddings[idx] =  nlp[word]
        return embeddings
      ## if word not in model then skip and that row stays all 0s  
      except:
        pass

# Function to filter out only relevant columns from dataframe
def filter_df(dataframe):
  try:
    dataframe = dataframe[train_columns]
    return dataframe
  except Exception as e:
    return print(e)

# Function to resample data
def resample(X, y):
  try:
    res = RandomUnderSampler(random_state=42)
    X_res, y_res = res.fit_resample(X,y)
    return X_res, y_res
  except Exception as e:
    return print(e)

In [211]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [212]:
# Select required input
suppliers = df[['VENDOR_NUMBER', 'VENDOR_NAME']]
        # pre-process textual data
suppliers['Cleaned_Name'] =preprocess_text(df['VENDOR_NAME'],remove_digits=True,lemm=True,eng=False)
        # reset index
suppliers.reset_index(inplace=True,drop=True)
sim=fuzz_similarity(suppliers['Cleaned_Name'])
clustered_vendor=company_clusters(suppliers, sim)
        # Generate standardized names
final=standard_name(clustered_vendor)

C:\Users\A427032\AppData\Local\Temp/ipykernel_27436/656536215.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suppliers['Cleaned_Name'] =preprocess_text(df['VENDOR_NAME'],remove_digits=True,lemm=True,eng=False)
C:\Program Files (x86)\Python\lib\site-packages\sklearn\cluster\_affinity_propagation.py:250: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  warnings.warn(


In [213]:
        # Generate standardized names
final=standard_name(clustered_vendor)
final

,VENDOR_NUMBER,VENDOR_NAME,Cleaned_Name,Cluster,StandardName,Score_with_standard
0,10517228,WILLIAN MARECO SANTACRUZ,willian mareco santacruz,-1,w,8
1,10387097,WILLIAN PEDRO FRANCO,willian pedro franco,-1,w,10
2,10444703,WILLIAN PIOVESAN ALMEIDA,willian piovesan almeida,-1,w,8
3,10417721,WILLIAN ROGER SCHLATTER,willian roger schlatter,-1,w,8
4,504674,WILLIE COOPER & SON,willie cooper son,-1,w,11
...,...,...,...,...,...,...
997,5222945,knip GmbH und Co.KG,knip gmbh und cokg,-1,w,0
998,57759,m2 commodities llc,m commodities,-1,w,0
999,57707,naimko Plastics & Trophy Center,naimko plastics trophy center,-1,w,0
1000,58052,steam valley farms,steam valley farms,-1,w,0
